# Training
This notebook demonstrates how to train a  model using the nnUNet framework. For more detailed instructions see `nnUNet/documentation/how_to_use_nnunet.md`.

## Sections:
1. Environment Setup
2. Hyperparameter Configuration
3. Model Training

## 1. Environment Setup

First, we set up the necessary environment variables that nnUNet uses to locate datasets and store results. If the nnUnet datasets were generated with the "preprocessing.ipynb" notebook, the default paths should work correctly automatically. Otherwise ensure these paths are correctly set according to your directory structure.

In [1]:
import os
from pathlib import Path
import time
import json

current_dir = Path.cwd()

# Default paths for the datasets
os.environ["nnUNet_raw"] = str(current_dir / "nnUNet_raw")
os.environ["nnUNet_preprocessed"] = str(current_dir / "nnUNet_preprocessed")
os.environ["nnUNet_results"] = str(current_dir / "nnUNet_results")

data_name = "Dataset101_FemurCorrected"

## 2. Data Preprocessing

The data preprocessing step ensures that the datasets are in the correct format and verifies their integrity. If using only one dataset with the default setup, this dataset will have the ID 101.

In [ ]:
# Preprocess Dataset 101 and verify its integrity
!nnUNetv2_plan_and_preprocess -d 101 --verify_dataset_integrity

# A different planner is needed for the usage of residual nets
# Here the medium residual network was chosen as it is the largest that still fits into 16GB VRAM
!nnUNetv2_plan_experiment -d 101 -pl nnUNetPlannerResEncM

## 2. Hyperparameter Configuration

Many parameters can be modified by utilizing the plans json files, see `nnUNet/documentation/explanation_plans_files.md` for more details. All different settings that were tried out in the report can be selected in the next code cell. The actual modified plans files are `nnUNet_preprocessed/{data_name}/nnUNetPlans.json` and `nnUNet_preprocessed/{data_name}/nnUNetResEncUNetMPlans.json`.
Parameters that are not covered in the plans files have to be set directly in the python files. For example, the number of training epochs in this report was changed from the default 1000 epochs to 100 epochs by changing the variable `num_epochs` in the file `nnUNet/nnunetv2/training/nnUNetTrainer/nnUNetTrainer.py`. This was only set during the parameter optimization in order to be able to try out a complete grid-search of all possible combinations of parameter values found in the code cell below.
The different data augmentation is chosen by selecting a different trainer file, for example the trainer for DA5 augmentation is found at `nnUNet/nnunetv2/training/nnUNetTrainer/variants/nnUNetTrainerDA5.py`.The mean validation Dice score for each configuration can be found at the end of their respecitve `training_log.txt` file.

In [ ]:
# Changing the plans files for the different configurations to work

file_path_1 = f"nnUNet_preprocessed/{data_name}/nnUNetPlans.json"
file_path_2 = f"nnUNet_preprocessed/{data_name}/nnUNetResEncUNetMPlans.json"

additional_config_1 = {
    "3d_fullresBS4": {
        "inherits_from": "3d_fullres",
        "batch_size": 4
    },
    "2dBS4": {
        "inherits_from": "2d",
        "batch_size": 744
    },
    "3d_fullresBS6": {
        "inherits_from": "3d_fullres",
        "batch_size": 6
    },
    "2dBS6": {
        "inherits_from": "2d",
        "batch_size": 1116
    }
}

additional_config_2 = {
    "3d_fullresBS3": {
        "inherits_from": "3d_fullres",
        "batch_size": 3
    },
    "2dBS3": {
        "inherits_from": "2d",
        "batch_size": 558
    },
    "3d_fullresBS4": {
        "inherits_from": "3d_fullres",
        "batch_size": 4
    },
    "3d_fullresBS6": {
        "inherits_from": "3d_fullres",
        "batch_size": 6
    },
    "2dBS4": {
        "inherits_from": "2d",
        "batch_size": 744
    }
}

def update_json_file(file_path, new_content):
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        if not all(key in data['configurations'] for key in new_content.keys()):
            data['configurations'].update(new_content)
            
            with open(file_path, 'w') as file:
                json.dump(data, file, indent=4)
            print(f"Updated {file_path} with new configurations.")
        else:
            print("The configurations are already present in the file.")
    else:
        print(f"The file {file_path} does not exist.")

update_json_file(file_path_1, additional_config_1)
update_json_file(file_path_2, additional_config_2)

In [2]:
# Dimensional structures
dim_structs = ["2D", "3D"]
chosen_dim_struct = dim_structs[1]

# Batch sizes, actual values are different depending on the dimensional as well as the network structure:
#   default network:
#       2D ... small = 372, medium = 744, large = 1116
#       3D ... small = 2, medium = 4, large = 6 
#   residual encoder network:
#       2D ... small = 377, medium = 558, large = 744
#       3D ... small = medium = 3, large = 4
batch_sizes = ["small", "medium", "large"]
chosen_batch_size = batch_sizes[2]

# Data augmentation methods
aug_methods = ["default", "DA5"]
chosen_aug_method = aug_methods[0]
    
# Encoder structure
network_structures = ["default", "ResEnc"]
chosen_network_structure = network_structures[0]

# choose the fold in a 5-fold cross validation-split ("all" trains the model on the whole training data)
folds = [0, 1, 2, 3, 4, "all"]
chosen_fold = folds[2]

## 3. Model Training

After chosing the desired parameters, run the next cell to start the training process and measure the needed time.
Training logs can be found under `nnUNet_results/{data_name}/{model_name}` .

In [3]:
if(chosen_aug_method=="default"):
    trainer= ""
if(chosen_aug_method=="DA5"):  
    trainer = " -tr nnUNetTrainerDA5"

if(chosen_dim_struct=="2D"):
    if(chosen_batch_size=="small"):
        if(chosen_network_structure=="default"):
            command = f"nnUNetv2_train 101 2d {chosen_fold}{trainer}"
        if(chosen_network_structure=="ResEnc"):
            command = f"nnUNetv2_train 101 2d {chosen_fold}{trainer} -p nnUNetResEncUNetMPlans"
    if(chosen_batch_size=="medium"):
        if(chosen_network_structure=="default"):
            command = f"nnUNetv2_train 101 2dBS4 {chosen_fold}{trainer}"
        if(chosen_network_structure=="ResEnc"):
            command = f"nnUNetv2_train 101 2dBS3 {chosen_fold}{trainer} -p nnUNetResEncUNetMPlans"
    if(chosen_batch_size=="large"):
        if(chosen_network_structure=="default"):
            command = f"nnUNetv2_train 101 2dBS6 {chosen_fold}{trainer}"
        if(chosen_network_structure=="ResEnc"):
            command = f"nnUNetv2_train 101 2dBS4 {chosen_fold}{trainer} -p nnUNetResEncUNetMPlans"
if(chosen_dim_struct=="3D"):
    if(chosen_batch_size=="small"):
        if(chosen_network_structure=="default"):
            command = f"nnUNetv2_train 101 3d_fullres {chosen_fold}{trainer}"
        if(chosen_network_structure=="ResEnc"):
            command = f"nnUNetv2_train 101 3d_fullres {chosen_fold}{trainer} -p nnUNetResEncUNetMPlans"
    if(chosen_batch_size=="medium"):
        if(chosen_network_structure=="default"):
            command = f"nnUNetv2_train 101 3d_fullresBS4 {chosen_fold}{trainer}"
        if(chosen_network_structure=="ResEnc"):
            command = f"nnUNetv2_train 101 3d_fullresBS3 {chosen_fold}{trainer} -p nnUNetResEncUNetMPlans"
    if(chosen_batch_size=="large"):
        if(chosen_network_structure=="default"):
            command = f"nnUNetv2_train 101 3d_fullresBS6 {chosen_fold}{trainer}"
        if(chosen_network_structure=="ResEnc"):
            command = f"nnUNetv2_train 101 3d_fullresBS4 {chosen_fold}{trainer} -p nnUNetResEncUNetMPlans"

print(command)
start_time = time.time()
!{command}
end_time = time.time()
elapsed_time = end_time - start_time    
elapsed_time

nnUNetv2_train 101 3d_fullresBS6 2

############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Traceback (most recent call last):
  File "/home/jere/.conda/envs/bones/bin/nnUNetv2_train", line 8, in <module>
    sys.exit(run_training_entry())
             ^^^^^^^^^^^^^^^^^^^^
  File "/home/jere/bones/nnUNet/nnunetv2/run/run_training.py", line 275, in run_training_entry
    run_training(args.dataset_name_or_id, args.configuration, args.fold, args.tr, args.p, args.pretrained_weights,
  File "/home/jere/bones/nnUNet/nnunetv2/run/run_training.py", line 196, in run_training
    nnunet_trainer = get_trainer_from_args(dataset_name_or_id, configuration, fold, trainer_class_name,
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

3.415040969848633